#Categorising the IMDB rating into 3 classes Hit,Avg,Flop
Here I have dataset named movie_metadata in which the target variable is IMDB score and other variables that decide the IMDB score. Instead of just IMDB score,With the help of other parameters I want to predict whether a movie is Hit,Avg or Flop.



|imdb_score | Classify |
| --- | ---|
|1-3 | Flop Movie|
|3-6 | Average Movie|
|6-10 | Hit Movie|


# 1 INTRODUCTION


## 1.1 Background


Success of a movie depends upon alot of factors like good directors or excellent actors or story plotline.However, famous directors and actors can always bring an expected box-office income but cannot guarantee a highly rated imdb score.



## 1.2 Describing Data

The dataset contains 28 variables for 5043 movies, spanning across 100 years in 66 countries. There are 2399 unique director names, and thousands of actors/actresses. “imdb_score” is the response variable while the other 27 variables are possible predictors.

|Variable Name |	Description|
| --- | --- |
|movie_title	 | Title of the Movie|
|duration	| Duration in minutes|
|director_name	| Name of the Director of the Movie|
|director_facebook_likes |	Number of likes of the Director on his Facebook Page|
|actor_1_name |	Primary actor starring in the movie|
|actor_1_facebook_likes |	Number of likes of the Actor_1 on his/her Facebook Page|
|actor_2_name |	Other actor starring in the movie|
|actor_2_facebook_likes	| Number of likes of the Actor_2 on his/her Facebook Page|
|actor_3_name |	Other actor starring in the movie|
|actor_3_facebook_likes |	Number of likes of the Actor_3 on his/her Facebook Page|
|num_user_for_reviews |	Number of users who gave a review|
|num_critic_for_reviews |	Number of critical reviews on imdb|
|num_voted_users | 	Number of people who voted for the movie|
|cast_total_facebook_likes |	Total number of facebook likes of the entire cast of the movie|
|movie_facebook_likes |	Number of Facebook likes in the movie page|
|plot_keywords |	Keywords describing the movie plot|
|facenumber_in_poster |	Number of the actor who featured in the movie poster|
|color |	Film colorization. ‘Black and White’ or ‘Color’|
|genres |	Film categorization like ‘Animation’, ‘Comedy’, ‘Romance’, ‘Horror’, ‘Sci-Fi’, ‘Action’, ‘Family’|
|title_year |	The year in which the movie is released (1916:2016)|
|language |	English, Arabic, Chinese, French, German, Danish, Italian, Japanese etc|
|country |	Country where the movie is produced|
|content_rating |	Content rating of the movie|
|aspect_ratio |	Aspect ratio the movie was made in|
|movie_imdb_link |	IMDB link of the movie|
|gross |	Gross earnings of the movie in Dollars|
|budget |	Budget of the movie in Dollars|
|imdb_score |	IMDB Score of the movie on IMDB|

Lets see which features influence the target varible(IMDB Score)

# 2 DATA EXPLORATION

## 2.1 Importing necessary Libraries


In [ ]:
#Importing necessary Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

In [ ]:
#Reading the dataset
data=pd.read_csv('../input/imdb-5000-movie-dataset/movie_metadata.csv')
data.head()

## 2.2 Categorizing the target varible 

Here we are categorizing the target variable in such a way that IMDB score between 1 and 3 is FLOP , between 3 and 6 is AVG, between 6 and 10 is HIT.

And we are using binning in pandas to acheive this.


In [ ]:
#Categorising the target varible 
bins = [ 1, 3, 6, 10]
labels = ['FLOP', 'AVG', 'HIT']
data['imdb_binned'] = pd.cut(data['imdb_score'], bins=bins, labels=labels)

 Barplot of imbd_binned column

In [ ]:
data.groupby(['imdb_binned']).size().plot(kind="bar",fontsize=14)
plt.xlabel('Categories')
plt.ylabel('Number of Movies')
plt.title('Categorization of Movies')

We can see a new column named imdb_binned correctly categorising the imdb score


In [ ]:
#Checking the new column
data.head(5)

Our dataset contains 5043 samples(rows) and 28 variables(columns)

In [ ]:
#Shape of the dataset
data.shape

## 2.3 Handling the Missing values

Every datset have some missing values, lets find out in which cloumns they are?

In [ ]:
#Total null values present in each column
data.isnull().sum()

Dropping all the samples that having missing values


In [ ]:
#Droping the samples that have missing values
data.dropna(inplace=True)

Total samples remaining after dropping missing values


In [ ]:
#Final shape of the data after Droping missing values
data.shape

In [ ]:
#List of variables in the datset
data.columns

In [ ]:
data.shape

Lets find out how the string variables are behaving

In [ ]:
#Describing the categorical data
data.describe(include='object')

'movie_title','movie_imdb_link' columns are almost unique,so they doesn't contribute in predicting target variable

In [ ]:
#Dropping 2 columns
data.drop(columns=['movie_title','movie_imdb_link'],inplace=True)

## 2.4 Label Encoding

All the categorical columns and the columns with text data are being Label Encodeded in this step.

In [ ]:
#Label encoding the categorical columns
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_list=['color', 'director_name', 'actor_2_name',
        'genres', 'actor_1_name',
        'actor_3_name',
        'plot_keywords',
        'language', 'country', 'content_rating',
       'title_year', 'aspect_ratio']
data[cat_list]=data[cat_list].apply(lambda x:le.fit_transform(x))

In [ ]:
#A sample of data after label encoding
data.head()

## 2.5 Correlation

To find out whether there is any relation between variables, in other terms multicollineariaty.



In [ ]:
#Finding Correlation between variables
corr = data.corr()
mask = np.zeros(corr.shape, dtype=bool)
mask[np.triu_indices(len(mask))] = True
plt.subplots(figsize=(20,15))
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns,cmap='RdYlGn',annot=True,mask = mask)

These variables that are correlated cause errors in the prediction, so removing them


In [ ]:
#Removing few columns due to multicollinearity
data.drop(columns=['cast_total_facebook_likes','num_critic_for_reviews'],inplace=True)

Removing the column "imdb_score" since we have "imdb_binned

I am gonna train the model with imdb_binned not with imdb_score so dropping the column.


In [ ]:
#Removing the column "imdb_score" since we have "imdb_binned"
data.drop(columns=['imdb_score'],inplace=True)

In [ ]:
data.shape

# 3 CLASSIFICATION MODEL BUILDING

Splitting the data into X and y where X contains Indepentent variables and y contain Target/Dependent variable.


In [ ]:
#Independent Variables
X = data.iloc[:, 0:23].values
#Dependent/Target Variable
y = data.iloc[:, 23].values
y

## 3.1 Train Test Split

We need data not only to train our model but also to test our model. So splitting the dataset into 70:30 (Train:Test) ratio.We have a predefined a function in Sklearn library called test_train_split, lets use that.

In [ ]:
#Spliting the data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0,stratify = y)
print(X_train.shape)
print(y_train.shape)

## 3.2 Scaling

Few variables will be in the range of Millions and some in Tens, lets bring all of them into same scale


In [ ]:
#Scaling the dependent variables
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## 3.4 Feature Selection using RFECV



Finding optimal features to use for Machine learning model training can sometimes be a difficult task to accomplish.There are just so many methods to choose from and here I am going with RFECV.

Recursive Feature Elimination  with Cross Validation

Recursive — involving doing or saying the same thing several times in order to produce a particular result or effect

Feature — individual measurable property or characteristic of a phenomenon being observed — an  attribute in your dataset

Cross-Validation — a technique for evaluating ML models by training several ML models on subsets of the available input data and evaluating them on the complementary subset of the data. Use cross-validation to detect overfitting, ie, failing to generalize a pattern.

You will need to declare two variables — X and y where first represents all the features, and the second represents the target variable. Then you’ll make an instance of the Machine learning algorithm (In this case RandomForests). In it, you can optionally pass a random state seed for reproducibility. Now you can create an instance of RFECV.




In [ ]:
#Performing Recursive Feauture Elimation with Cross Validation
#Using Random forest for RFE-CV and logloss as scoring
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
clf_rf=RandomForestClassifier(random_state=0)
rfecv=RFECV(estimator=clf_rf, step=1,cv=5,scoring='neg_log_loss')
rfecv=rfecv.fit(X_train,y_train)

In [ ]:
#Optimal number of features
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
print('Optimal number of features :', rfecv.n_features_)
print('Best features :', X_train.columns[rfecv.support_])

|Features Selected |	Features Dropped|
| --- | --- |
|duration| color|
|director_facebook_likes	| director name|
|actor_3_facebook_likes	| actor_2_name|
|actor_1_facebook_likes|	actor_1_name   |
|gross|	facenumber_in_poster|
|genres |	language|
|num_voted_users |country	|
|actor_3_name 	| content_rating|
|actor_3_name |	aspect_ratio|
|plot_keywords |	|
|num_user_for_reviews |	 |
|budget| |
|title_year | 	|
|actor_2_facebook_likes |	 |
|movie_facebook_likes |	 |


In [ ]:
#Feauture Ranking
clf_rf = clf_rf.fit(X_train,y_train)
importances = clf_rf.feature_importances_

std = np.std([tree.feature_importances_ for tree in clf_rf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]


In [ ]:
#Logloss vs Number of features
import matplotlib.pyplot as plt
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score of number of selected features")
plt.title("Log loss vs Number of fetures")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

In [ ]:
#Selecting the Important Features
X_opt = X_train.iloc[:,X_train.columns[rfecv.support_]]
X_test = X_test.iloc[:,X_test.columns[rfecv.support_]]

In [ ]:
#Creating anew dataframe with column names and feature importance
dset = pd.DataFrame()
data1 = data
data1.drop(columns=['imdb_binned'],inplace=True)
dset['attr'] = data1.columns

dset['importance'] = clf_rf.feature_importances_
#Sorting with importance column
dset = dset.sort_values(by='importance', ascending=True)

#Barplot indicating Feature Importance
plt.figure(figsize=(16, 14))
plt.barh(y=dset['attr'], width=dset['importance'], color='#1976D2')
plt.title('RFECV - Feature Importances', fontsize=20, fontweight='bold', pad=20)
plt.xlabel('Importance', fontsize=14, labelpad=20)
plt.show()

## 3.4 Random Forest

Random forests is an ensemble learning method for classification that operate by constructing a multitude of decision trees at training time and outputting the class that is the mode of the classes (classification)  of the individual trees

*n_estimators* is a parameter that specify number of trees in the forest.

*criterion* is to specify what function to measure the quality of a split. “entropy” is for the information gain. 

In [ ]:
#Training the Random Forest Classifer on Train data
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)
classifier.fit(X_opt, y_train)


Predicting the test data

In [ ]:
#Predicting the target variable
y_pred = classifier.predict(X_test)

## 3.5 Confusion Matrix

Confusion matrix gives a clear view of ground truth and prediction.

In [ ]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix 
cm = confusion_matrix(y_test,y_pred)
cm

## 3.6 Classification Report

In [ ]:
#Classification Report
from sklearn.metrics import classification_report
cr = classification_report(y_test,y_pred)
print(cr)